In [28]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, clear_output
import random
import time
from itertools import combinations
import requests
from bs4 import BeautifulSoup
import json

%matplotlib inline

In [29]:
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')

    if content is None:
        print("Script tag not found. Data not uploaded.")
        return False

    content_text = content.text[27:-1] if content.text else None

    if not content_text:
        print("Content text not found. Data not uploaded.")
        return False

    try:
        result = json.loads(content_text)[1][1]
        print("Content of the script tag:")
        print(result)
        form_dict = {}
    
        loaded_all = True
        for item in result:
            # It is used to remove non-alphanumeric characters from the item name
            item_name_cleaned = ''.join(char.lower() for char in item[1] if char.isalnum())
            
            # Check if the cleaned item name matches any key in the data dictionary
            matching_keys = [key for key in data_dict.keys() if item_name_cleaned in ''.join(char.lower() for char in key if char.isalnum())]
            
            if not matching_keys:
                print(f"Form item {item[1]} not found. Data not uploaded.")
                loaded_all = False
                return False
            
            # Use the first matching key as the data dictionary key
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[matching_keys[0]]
        
        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok
    
    except (json.JSONDecodeError, IndexError) as e:
        print(f"Error decoding JSON or accessing result: {e}")
        return False
            

In [30]:
def draw_cubes(cubes, ticks=False, grid=False, view='', flip='', rot=0, ax3d=None):
# make figure and 3d axes for plotting
    if ax3d is None:
        # Create a figure and 3D axes for plotting
        fig = plt.figure()
        ax = fig.add_subplot(111,projection='3d')
    else:
        ax = ax3d
     # The cubes can be plotted using a 3D voxels plot
    ax.voxels(cubes != '', facecolors=cubes, edgecolors='k', shade=False)
    

    nx, ny, nz = cubes.shape

    ax.axes.set_xlim3d(0, nx) 
    ax.axes.set_ylim3d(0, ny) 
    ax.axes.set_zlim3d(0, nz) 
    
    # view argument allows users to set a 2D projection
    if view == 'xy': ax.view_init(90, -90, 0+rot)
    elif view == '-xy': ax.view_init(-90, 90, 0-rot)
    elif view == 'xz': ax.view_init(0, -90, 0+rot)
    elif view == '-xz': ax.view_init(0, 90, 0-rot)
    elif view == 'yz': ax.view_init(0, 0, 0+rot)
    elif view == '-yz': ax.view_init(0, 180, 0-rot)
    else:   ax.view_init(azim=ax.azim+rot)

    # flip argument allows user to show a mirror image
    # flip='x' reverses image in x direction etc.
    if 'x' in flip: ax.axes.set_xlim3d(nx, 0) 
    if 'y' in flip: ax.axes.set_ylim3d(ny, 0) 
    if 'z' in flip: ax.axes.set_zlim3d(nz, 0) 

    # style figure ticks and grid lines
    if ticks==False: 
        for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
            axis.set_ticklabels([])
            axis.line.set_linestyle('')
            axis._axinfo['tick']['inward_factor'] = 0.0
            axis._axinfo['tick']['outward_factor'] = 0.0
            
    if grid==False and ticks==False: ax.set_axis_off()
    
    if ax3d is not None:
        # return axes with result
        return
    else:
        # show image
        display(fig)
        # delete figure
        plt.close(fig)
    return

In [31]:
def quiz_cubes (number_of_colors):
    # Set up question cube and the cube for the wrong projection
    # For the cubes3
        # Define possible colors for the original cube
        colors = random.sample(possible_colors, number_of_colors)
        # Randomly insert color codes and block sizes into the original cube
        for color in colors:
            x, y, z = np.random.randint(5, size=3)
            size_x, size_y, size_z = np.random.randint(1, 4, size=3)  # Random block sizes
            cubes3[x:x+size_x, y:y+size_y, z:z+size_z] = color
        # For the new_cube
        # Copy the colors from cubes3 to new_cube
        for color in np.unique(cubes3):
            if color in possible_colors:
                new_cube[cubes3 == color] = color
        # Randomly insert new block sizes into the new cube
        for color in np.unique(new_cube):
            if color in possible_colors:
                x, y, z = np.random.randint(5, size=3)
                size_x, size_y, size_z = np.random.randint(1, 4, size=3)  
                new_cube[new_cube == color] = color  # Keep the color the same
                new_cube[x:x+size_x, y:y+size_y, z:z+size_z] = color  # Update with random block sizes   
        return


In [45]:
# Set up the quiz
choices = ['a', 'b', 'c', 'd']
quiz_questions = []
mark = 0
quiz_start_time = time.time()
quiz_duration = 180  # 3 minutes
elapsed_time1 = 0
quiz_number = 0
incorrect_question = []

# To make sure questions are same for the test each time
global_seed = 42
np.random.seed(global_seed)
random.seed(global_seed)

# Set up cubes
for i in range(1000): 
    # Define a 3D 5x5x5 string array with entries set to ''
    cubes3 = np.full((5, 5, 5), '')
    # For the correct answer, create a new cube with similar colors but random block sizes
    new_cube = np.full_like(cubes3, '')  # Initialize with the same shape and dtype as cubes3
    
    # Define colors
    possible_colors = ['r', 'g', 'b', 'm', 'y']

    # Creating a Progressive Difficulty Gradient
    if 0<= i <=1:
        quiz_cubes (2)  
    elif 1< i <= 4:
        quiz_cubes (3) 
                    
    elif 4 < i <= 8:
        quiz_cubes (4) 
    else:
       quiz_cubes (5) 

 # Dictionary to store projections for correct and incorrect answers
    projections = {
        correct_answer: '',
        incorrect_choices[0]: '',
        incorrect_choices[1]: '',
        incorrect_choices[2]: ''
    }
    
    # Add the question to the quiz
    quiz_questions.append({
        'cubes': cubes3,
        'projections': projections,
        'correct_answer': correct_answer,
        'incorrect_answer': incorrect_choices,
        'new_cube': new_cube
    })

# Record the result
results = []
form_url = "https://docs.google.com/forms/d/e/1FAIpQLScwJovnnWunq0VFFKiGpCo29yXme-wYTkIUzELAUejT4i7FEQ/viewform?usp=sf_link"   
      
# Quiz

# The data disclaimer question
print("Please read:")
print("")
print("we wish to record your response data")
print("to an anonymised public data repository. ")
print("Your data will be used for educational teaching purposes")
print("practising data analysis and visualisation.")
print("")
print("Please type   yes   in the box below if you consent to the upload.")
result = input("> ")
if result == "yes":
    print("Thanks - your data will be uploaded.")
    #send_to_google_form(data_dict, form_url)

else:
    print("No problem we hope you enjoyed the test.")

# Quiz started
print("This test will last 3 minutes, please try your best to do more questions!")
print("Please enter your username or id:")
ans1 = input("> ")

print("Please enter your gender:")
ans2 = input(">")

print("Please enter your age:")
ans3 = input("> ")

for i, question in enumerate(quiz_questions, 1):
    
    print(f"Question {i}:")
    question_number = str(i)
    draw_cubes(question['cubes'], grid=True)
    print("The following is the view after rotating 180")
    draw_cubes(question['cubes'],rot = 180, grid=True)
    print("Choose the 2D projection that cannot be made by rotating the arrangement in space:")

    possible_projections = ['xy', '-xy', 'xz', '-xz', 'yz', '-yz']
    
    # Choose a random correct answer
    correct_answer = random.choice(choices)
    # The list of incorrect options by removing the correct answer
    incorrect_choices = [choice for choice in choices if choice != correct_answer]

    # Generate incorrect projections
    for i in range(3):
        incorrect_projection = random.choice(possible_projections)
        possible_projections.remove(incorrect_projection)
        projections[incorrect_choices[i]] = incorrect_projection

    # Record the time
    start_time = time.time()
    
    # Creat content in options
    for choice in choices:
        if choice in incorrect_choices:
            print(f"Answer {choice} is: ")
            draw_cubes(question['cubes'], view=projections[choice])
        else: # For correct answer
            correct_projection = random.choice(possible_projections)
            projections[correct_answer] = correct_projection
            print(f"Answer {choice} is:")
            draw_cubes(question['new_cube'], view=projections[correct_answer])

    # Calculate the elaped time
    elapsed_time = time.time() - quiz_start_time

    user_answer = input("Your Answer (a, b, c, or d): ").lower()

    clear_output(wait=True)

    # correct answer or not
    if user_answer == correct_answer:
        print("Correct! Well done.")
        mark +=1
        print(f"You spent {elapsed_time - elapsed_time1:.2f} seconds on this problem")
        print(f"Your time left:{quiz_duration - elapsed_time:.2f} seconds ")
        elapsed_time1 = elapsed_time
    else:
        print(f"Sorry, that's incorrect. The correct answer is {question['correct_answer']}.")
        draw_cubes(question['new_cube'], view=projections[correct_answer])
        print(f"You spent {elapsed_time - elapsed_time1:.2f} seconds on this problem")
        print(f"Your time left:{quiz_duration - elapsed_time:.2f} seconds ")
        elapsed_time1 = elapsed_time
        incorrect_question.append(question_number)
       
    clear_output(wait=True)
    quiz_number +=1
    
    # Interval between questions
    input("Press Enter to continue to the next question...")

    # Stop the quiz
    if elapsed_time > quiz_duration:
        clear_output(wait=True)
        print(f"Time is up")
        break
        
print(f"Congratulations! You've completed the quiz, and your mark is {mark}")

# Data to collect
time_spent = int(elapsed_time)/quiz_number
print(int(elapsed_time))
print(quiz_number)
result_dict  = {
    'result': result,
    'Name': ans1,
    'Gender': ans2,
    'Age': ans3,
    'Number of questions answered': quiz_number,
    'Scores': mark,
    'Average time spent per question': time_spent,
    'Incorrect question': incorrect_question
    }

# Upload the result to the Google Form
if result == "yes":
    upload_success = send_to_google_form(result_dict, form_url)
else:
    upload_success = send_to_google_form({'result': result}, form_url)

# Print the result
if upload_success:
    print("Data uploaded successfully!")
else:
    print("Data upload failed.")

Time is up
Congratulations! You've completed the quiz, and your mark is 7
210
15
Content of the script tag:
[[918026115, 'result', None, 0, [[271092970, None, 0]], None, None, None, None, None, None, [None, 'result<br>']], [2052849999, 'Name', None, 0, [[675332650, None, 0]], None, None, None, None, None, None, [None, 'Name']], [104833321, 'Gender', None, 0, [[2029009840, None, 0]], None, None, None, None, None, None, [None, 'Gender']], [338098159, 'Age', None, 0, [[1647652781, None, 0]], None, None, None, None, None, None, [None, 'Age']], [1798681343, 'Number of questions answered', None, 0, [[1755964194, None, 0]], None, None, None, None, None, None, [None, 'Number of questions answered<br>']], [1453446443, 'Scores', None, 0, [[1142154393, None, 0]], None, None, None, None, None, None, [None, 'Scores']], [170299733, 'Average time spent per question', None, 0, [[288935677, None, 0]], None, None, None, None, None, None, [None, 'Average time spent per question<br>']], [353721375, 'Incor